In [295]:
# %matplotlib inline
# import mpld3
# mpld3.enable_notebook()
import rosbag_pandas
import pandas as pd
import matplotlib.pyplot as plt
import easygui
import rosbag
import json_tricks as json
import numpy as np
import cPickle as pkl
import time
import seaborn as sns
def TicTocGenerator():
    # Generator that returns time differences
    ti = 0           # initial time
    tf = time.time() # final time
    while True:
        ti = tf
        tf = time.time()
        yield tf-ti # returns the time difference

TicToc = TicTocGenerator() # create an instance of the TicTocGen generator

# This will be the main function through which we define both tic() and toc()
def toc(tempBool=True):
    # Prints the time difference yielded by generator instance TicToc
    tempTimeInterval = next(TicToc)
    if tempBool:
        print( "Elapsed time: %f seconds.\n" %tempTimeInterval )

def tic():
    # Records a time in TicToc, marks the beginning of a time interval
    toc(False)
    
def pickler( obj, path):
    """
    Pickle a Python object
    """
    with open(path, "wb") as pfile:
        pkl.dump(obj, pfile)

def depickler( path):
    """
    Extracts a pickled Python object and returns it
    """
    with open(path, "rb") as pfile:
        data = pkl.load(pfile)
    return data
def circles(x, y, s, c='b', vmin=None, vmax=None, **kwargs):
    """
    Make a scatter of circles plot of x vs y, where x and y are sequence 
    like objects of the same lengths. The size of circles are in data scale.

    Parameters
    ----------
    x,y : scalar or array_like, shape (n, )
        Input data
    s : scalar or array_like, shape (n, ) 
        Radius of circle in data unit.
    c : color or sequence of color, optional, default : 'b'
        `c` can be a single color format string, or a sequence of color
        specifications of length `N`, or a sequence of `N` numbers to be
        mapped to colors using the `cmap` and `norm` specified via kwargs.
        Note that `c` should not be a single numeric RGB or RGBA sequence 
        because that is indistinguishable from an array of values
        to be colormapped. (If you insist, use `color` instead.)  
        `c` can be a 2-D array in which the rows are RGB or RGBA, however. 
    vmin, vmax : scalar, optional, default: None
        `vmin` and `vmax` are used in conjunction with `norm` to normalize
        luminance data.  If either are `None`, the min and max of the
        color array is used.
    kwargs : `~matplotlib.collections.Collection` properties
        Eg. alpha, edgecolor(ec), facecolor(fc), linewidth(lw), linestyle(ls), 
        norm, cmap, transform, etc.

    Returns
    -------
    paths : `~matplotlib.collections.PathCollection`

    Examples
    --------
    a = np.arange(11)
    circles(a, a, a*0.2, c=a, alpha=0.5, edgecolor='none')
    plt.colorbar()

    License
    --------
    This code is under [The BSD 3-Clause License]
    (http://opensource.org/licenses/BSD-3-Clause)
    """
    import numpy as np
    import matplotlib.pyplot as plt
    from matplotlib.patches import Circle
    from matplotlib.collections import PatchCollection

    if np.isscalar(c):
        kwargs.setdefault('color', c)
        c = None
    if 'fc' in kwargs: kwargs.setdefault('facecolor', kwargs.pop('fc'))
    if 'ec' in kwargs: kwargs.setdefault('edgecolor', kwargs.pop('ec'))
    if 'ls' in kwargs: kwargs.setdefault('linestyle', kwargs.pop('ls'))
    if 'lw' in kwargs: kwargs.setdefault('linewidth', kwargs.pop('lw'))

    patches = [Circle((x_, y_), s_) for x_, y_, s_ in np.broadcast(x, y, s)]
    collection = PatchCollection(patches, **kwargs)
    if c is not None:
        collection.set_array(np.asarray(c))
        collection.set_clim(vmin, vmax)

    ax = plt.gca()
    ax.add_collection(collection)
    ax.autoscale_view()
    if c is not None:
        plt.sci(collection)
    return collection

In [173]:
dff['metadata']['parameters']

TypeError: 'NoneType' object has no attribute '__getitem__'

In [300]:
import matplotlib as mpl

def quadPlotter(df=df,metadata=metadata,off=20):
    
    parameters=metadata['parameters']
        # row and column sharing
    fig, ((ax2, ax1), (ax3, ax4)) = plt.subplots(2, 2, sharex=True, sharey=True,figsize=(16,16))
    ax={4:ax4,1:ax1,2:ax2,3:ax3}
    maxS=df.trajectory__speed.max()
    # ax1.set_title('lr anal')
    for i in range(int(df.trajectory__case.max()+1)):
        df2=df[(df.trajectory__case==i) & (df.trajectory__runNum<20) &(df.trajectory__isFlying==True)]
        im=ax[i+1].scatter(df2.trajectory__pPos_x,df2.trajectory__pPos_y,lw=0,alpha=1,s=5
                        ,c=df2.trajectory__speed.values/maxS,cmap='viridis')
        
        
#         ax[i+1].scatter(df2.trajectory__o1Pos_x,df2.trajectory__o1Pos_y,c='g',s=parameters['obj1Scale']
# )
#         ax[i+1].scatter(df2.trajectory__o2Pos_x,df2.trajectory__o2Pos_y,c='r',s=parameters[ 'obj2Scale']
# )
#         ax[i+1].
        circles(df2.trajectory__o1Pos_x[0],df2.trajectory__o1Pos_y[0],c='g',s=parameters['obj1Scale'])
        ax[i+1].set_xlim([513-off,513+off])
        ax[i+1].set_ylim([513-off,513+off])

        ax[i+1].set_aspect('equal', 'datalim')
        ax[i+1].set_title(i)
        print i
    
#     # Colorbar
#     ax3.cax.colorbar(ax3)
#     ax3.cax.toggle_label(True)
#     fig.colorbar(im, ax=ax3)
    
#     cax,kw = mpl.colorbar.make_axes([ax1,ax2,ax3,ax4])
#     plt.colorbar(im, cax=cax, **kw)
    plt.tight_layout()

    fig.subplots_adjust(right=0.8)

    cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
    fig.colorbar(im, cax=cbar_ax)

    plt.show()
    return fig

def loader():
    '''Load a dataframe from pickle and analyse lr cases'''

    defaultPath="/home/rhagoletis/catkin/src/World/bags/"
    path=easygui.fileopenbox(title="Choose pickle Dataframes"
                              ,default=defaultPath,
                              multiple=False,filetypes=["*.pickle"])
    print path

    dff=pd.read_pickle(path)
    df=dff['df']
    metadata=dff['metadata']
    return df,metadata


In [294]:
parameters=metadata['parameters']
parameters[ 'obj2Scale']


4.0

In [301]:
# df,metadata=loader()
quadPlotter(df,metadata,3)

0
1
2
3


In [226]:
# row and column sharing
f, ((ax2, ax1), (ax3, ax4)) = plt.subplots(2, 2, sharex=True, sharey=True,figsize=(16,16))
ax={4:ax4,1:ax1,2:ax2,3:ax3}
# ax1.set_title('lr anal')
for i in range(int(df.trajectory__case.max()+1)):
    df2=df[(df.trajectory__case==i) & (df.trajectory__runNum<20) &(df.trajectory__isFlying==True)]
    ax[i+1].scatter(df2.trajectory__pPos_x,df2.trajectory__pPos_y,lw=0,alpha=1,s=0.2,c=(df2.trajectory__speed,df2.trajectory__speed,1))
    ax[i+1].set_aspect('equal', 'datalim')
    ax[i+1].set_title(i)
    print i
plt.tight_layout()
plt.show()



ValueError: to_rgba: Invalid rgba arg "2017-01-04 17:00:11.976358912    0.644993
2017-01-04 17:00:11.982251008    0.629798
2017-01-04 17:00:11.988240896    0.629798
2017-01-04 17:00:11.994285056    0.553824
2017-01-04 17:00:12.000350976    0.553824
2017-01-04 17:00:12.006448128    0.553824
2017-01-04 17:00:12.012485888    0.523435
2017-01-04 17:00:12.018488832    0.523435
2017-01-04 17:00:12.024570112    0.660188
2017-01-04 17:00:12.030556928    0.660188
2017-01-04 17:00:12.036623872    0.690577
2017-01-04 17:00:12.042733056    0.614603
2017-01-04 17:00:12.048837888    0.614603
2017-01-04 17:00:12.054867968    0.736161
2017-01-04 17:00:12.060924928    0.736161
2017-01-04 17:00:12.067065088    0.538630
2017-01-04 17:00:12.073092864    0.690577
2017-01-04 17:00:12.079116032    0.690577
2017-01-04 17:00:12.085275904    0.477851
2017-01-04 17:00:12.091328000    0.493046
2017-01-04 17:00:12.097352960    0.493046
2017-01-04 17:00:12.103454208    0.599409
2017-01-04 17:00:12.109474816    0.59

In [278]:
'''Load a dataframe from pickle and analyse lr cases'''

defaultPath="/home/rhagoletis/catkin/src/World/bags/"
path=easygui.fileopenbox(title="Choose pickle Dataframes"
                          ,default=defaultPath,
                          multiple=False,filetypes=["*.pickle"])
print path

dff=pd.read_pickle(path)
df=dff['df']
dff

None


TypeError: coercing to Unicode: need string or buffer, NoneType found

In [81]:
print df2.trajectory__o1Pos_x[0],df2.trajectory__o1Pos_y[0]
print df2.trajectory__o2Pos_x[0],df2.trajectory__o2Pos_y[0]

0.0 0.0
0.0 0.0


In [72]:
df2.trajectory__o1Pos_x.plot()
df2.trajectory__o2Pos_x.plot()
df2.trajectory__o1Pos_y.plot()
df2.trajectory__o2Pos_y.plot()
plt.show()

In [96]:
df2=df[df.trajectory__case==3]
# df2

In [97]:
plt.scatter(df2.trajectory__pPos_x,df2.trajectory__pPos_y,lw=0)
plt.show()

In [106]:
range(3)

[0, 1, 2]

In [85]:
df.trajectory__case.max()

3.0

In [66]:
jsonPath=paths[0].split('.bag')[0]+'.json'
jsonPath

'/home/rhagoletis/catkin/src/World/bags/2017_01_02/2017-01-02__15:45:54_apple11_01_traj.json'

In [44]:
with open(jsonPath,'r') as js:
    a= js.read()

In [46]:
json.loads(a)

ValueError: No JSON object could be decoded